In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score # 정확도 함수
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [4]:
windata = pd.read_csv('../Dataset/win/Grandmaster.csv')
losedata = pd.read_csv('../Dataset/lose/Grandmaster_lose.csv')

In [5]:
data = pd.concat([windata, losedata])

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']
display(data)

,matchId,queueId,Diff_LV,Diff_CS,Diff_jglCS,Diff-K,Diff-K-top,K-WIN-top,K-LOSE-top,Diff-K-jug,...,WIN_controlWARDPlaced,Diff_WARDkill,Diff_Inhibitor,Diff_TOWERkill,Diff_FirstDRAGON,Diff_FirstHERALD,Diff_Firsttower,Diff_FirstBLOOD,dragonType,result
0,KR_6741690974,420,-1,43,12,1,-2,1,3,0,...,11,-1,0,-1,1,-1,-1,1,6,1
1,KR_6745383896,420,2,14,-16,1,1,1,0,2,...,13,3,0,1,1,1,1,1,5,1
2,KR_6744402678,420,2,75,-16,6,2,4,2,2,...,12,4,0,2,1,1,1,1,3,1
3,KR_6742865044,420,2,-22,24,0,2,3,1,-2,...,5,4,0,-3,1,-1,-1,-1,4,1
4,KR_6749510736,420,7,100,-4,10,4,4,0,3,...,8,-3,0,0,1,-1,1,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6348,KR_6750734852,420,1,-6,24,-5,-2,6,4,-1,...,13,-7,0,0,-1,1,1,1,1,0
6349,KR_6748369427,420,-1,-22,-22,1,1,1,2,-2,...,7,7,0,0,1,-1,0,1,6,0
6350,KR_6741044210,420,-1,-39,5,-2,1,1,2,0,...,9,-1,0,-2,1,-1,-1,-1,5,0
6351,KR_6746529749,420,-3,-73,-1,-7,-1,2,1,-3,...,8,-12,0,0,-1,1,-1,1,1,0


In [4]:
# 드래곤 타입 리스트
data['AIR_DRAGON_Type'] = np.where((data['Diff_FirstDRAGON'] == 1) & (data['dragonType'] == 1), 1, 0)
data['EARTH_DRAGON_Type'] = np.where((data['Diff_FirstDRAGON'] == 1) & (data['dragonType'] == 2), 1, 0)
data['FIRE_DRAGON_Type'] = np.where((data['Diff_FirstDRAGON'] == 1) & (data['dragonType'] == 3), 1, 0)
data['WATER_DRAGON_Type'] = np.where((data['Diff_FirstDRAGON'] == 1) & (data['dragonType'] == 4), 1, 0)
data['HEXTECH_DRAGON_Type'] = np.where((data['Diff_FirstDRAGON'] == 1) & (data['dragonType'] == 5), 1, 0)
data['CHEMTECH_DRAGON_Type'] = np.where((data['Diff_FirstDRAGON'] == 1) & (data['dragonType'] == 6), 1, 0)

# 불필요한 기존 칼럼 삭제
data = data.drop(['dragonType'], axis=1)

display(data)

,matchId,queueId,Diff_LV,Diff_CS,Diff_jglCS,Diff-K,Diff-K-top,K-WIN-top,K-LOSE-top,Diff-K-jug,...,Diff_FirstHERALD,Diff_Firsttower,Diff_FirstBLOOD,result,AIR_DRAGON_Type,EARTH_DRAGON_Type,FIRE_DRAGON_Type,WATER_DRAGON_Type,HEXTECH_DRAGON_Type,CHEMTECH_DRAGON_Type
0,KR_6741690974,420,-1,43,12,1,-2,1,3,0,...,-1,-1,1,1,0,0,0,0,0,1
1,KR_6745383896,420,2,14,-16,1,1,1,0,2,...,1,1,1,1,0,0,0,0,1,0
2,KR_6744402678,420,2,75,-16,6,2,4,2,2,...,1,1,1,1,0,0,1,0,0,0
3,KR_6742865044,420,2,-22,24,0,2,3,1,-2,...,-1,-1,-1,1,0,0,0,1,0,0
4,KR_6749510736,420,7,100,-4,10,4,4,0,3,...,-1,1,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6348,KR_6750734852,420,1,-6,24,-5,-2,6,4,-1,...,1,1,1,0,0,0,0,0,0,0
6349,KR_6748369427,420,-1,-22,-22,1,1,1,2,-2,...,-1,0,1,0,0,0,0,0,0,1
6350,KR_6741044210,420,-1,-39,5,-2,1,1,2,0,...,-1,-1,-1,0,0,0,0,0,1,0
6351,KR_6746529749,420,-3,-73,-1,-7,-1,2,1,-3,...,1,-1,1,0,0,0,0,0,0,0


In [8]:
def save_win_dataframe_to_csv(dataframe, filename):
        df = pd.DataFrame(dataframe)
        df.to_csv(filename, index=False)

In [9]:
save_win_dataframe_to_csv(data,'../Dataset/win/Grandmaster_Drangon.csv')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = ExtraTreesClassifier(n_estimators=100, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

#드래곤 전처리 후 정확도 79.5프로(그랜드마스터)
#오리지널 정확도 79.2(그랜드마스터)

정확도: 0.7925716084356311
